In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, explode
from pyspark.sql.types import StringType
from pyspark.ml.feature import Tokenizer, StopWordsRemover
import os


spark = SparkSession.builder.appName("ResumenLibros").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/09 00:11:22 WARN Utils: Your hostname, Gyro, resolves to a loopback address: 127.0.1.1; using 172.19.134.43 instead (on interface eth0)
25/12/09 00:11:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/09 00:11:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
tokenizer = Tokenizer(inputCol="clean", outputCol="tokens")
remover = StopWordsRemover(inputCol="tokens", outputCol="filtered")

### Limpia los nombres de archivo y hace el resumen del libro

In [2]:
df_tfidf = spark.read.parquet("tfidf_vectors.parquet")

# Extraer solo el nombre del archivo
def extraer_nombre(ruta):
    return os.path.basename(ruta).replace("%20", " ")

extraer_nombre_udf = udf(extraer_nombre, StringType())

df_tfidf = df_tfidf.withColumn("libro", extraer_nombre_udf(col("filename")))

# Función de resumen usando TF-IDF
def resumen_libro(libro, top_n=20):
    fila = df_tfidf.filter(col("libro") == libro).first()

    if fila is None:
        return f"No se encontró el libro '{libro}' en tfidf_vectors.parquet"

    vector = fila.tfidf
    valores = vector.toArray().tolist()

    indices_ordenados = np.argsort(valores)[::-1][:top_n]

    palabras = [f"feature_{i}" for i in indices_ordenados]

    return " ".join(palabras)

### Busca el libro especificado y muestra el resumen

In [3]:
libro_ejemplo = "Dracula.txt"

resumen = resumen_libro(libro_ejemplo)
print(f"Resumen de '{libro_ejemplo}' en 20 palabras:")
print(resumen)

[Stage 1:>                                                          (0 + 1) / 1]

Resumen de 'Dracula.txt' en 20 palabras:
feature_795 feature_12193 feature_11925 feature_11150 feature_12401 feature_15273 feature_8229 feature_19556 feature_12322 feature_17588 feature_17703 feature_12502 feature_5363 feature_5782 feature_3057 feature_10552 feature_8500 feature_3100 feature_16464 feature_11521


----------------------------------------                                        
Exception occurred during processing of request from ('127.0.0.1', 42002)
Traceback (most recent call last):
  File "/usr/lib/python3.12/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.12/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.12/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.12/socketserver.py", line 761, in __init__
    self.handle()
  File "/home/robc/SistDist/env/lib/python3.12/site-packages/pyspark/accumulators.py", line 299, in handle
    poll(accum_updates)
  File "/home/robc/SistDist/env/lib/python3.12/site-packages/pyspark/accumulators.py", line 271, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/home/robc/SistDist/